In [1]:
import numpy as np

# Problem description: The robot aims to clean the dusts in room D.
# The layout of the rooms are shown below.
# --------------
# |  A  |  B   |
# --------------
# |  C  | .D.  |
# --------------

# Define states
states = [
    's1',  # at A, D uncleaned
    's2',  # at B, D uncleaned
    's3',  # at C, D uncleaned
    's4',  # at D, D uncleaned
    's5'   # at D, D cleaned
]

# Terminal state is s5, where D is cleaned
terminal_states = ['s5']

# Define actions
actions = ['left', 'right', 'up', 'down', 'suck']

# Applicable actions for each state, except terminal states
applicable_actions = [
    ['right', 'down', 'suck'],  # for state s1
    ['left', 'down', 'suck'],  # for state s2
    ['right', 'up', 'suck'],  # for state s3
    ['left', 'up', 'suck']  # for state s4
]

# Next state when applying each applicable action in each state
next_states = [
    ['s2', 's3', 's1'],   # s'(s1, a)
    ['s1', 's4', 's2'],   # s'(s2, a)
    ['s4', 's1', 's3'],   # s'(s3, a)
    ['s3', 's2', 's5']   # s'(s4, a)
]

# Reward when applying each applicable action in each state
rewards = [
    [-0.04, -0.04, -0.1],   # r(s1, a)
    [-0.04, -0.04, -0.1],   # r(s2, a)
    [-0.04, -0.04, -0.1],   # r(s3, a)
    [-0.04, -0.04, -0.1]   # r(s4, a)
]

# Discount factor
gamma = 0.9

# When reaching the terminal state s5, we receive a reward of 1.0.
terminal_reward = 1.0

###
# Policy evaluation
###

"""
This policy evaluation method takes a policy 
and returns the state values for all the states under this policy, 
except the terminal states.
"""
def policy_evaluation(policy):
    v = {} # initialise a dictionary 'v' to map states to state values
    for state in states:
        # foreach terminal state s_T, initialise its state value as its predefined reward v_π(s_T) = r_sT
        if state in terminal_states: 
            v[state] = terminal_reward
        # foreach state, set its state value v_π(s) to 0
        else: 
            v[state] = 0
            
    # Iterate until convergence
    while True:
        delta = 0 # value gap ∆ = 0
        # iterate over each state (except terminal states)
        for i, state in enumerate(states[:-1]): # excluding last index, which is terminal state s5
            v_old = v[state] # copying old state value to v_old
            
            # Updating state value: finding all the calculation components
            action = policy[state] # Get the action for the state from the policy dictionary
            action_index = applicable_actions[i].index(action) # find the index of that action in the applicable_actions list
            next_state = next_states[i][action_index] # Get the next state for the current (state, action) pair
            reward = rewards[i][action_index] # Get the reward for the current (state, action) pair

            # Calculate the updated the state value: v_π(s) = r_(s,π(s)) + γ ∗ v_π(s′_(s,π(s)))
            v[state] = reward + gamma * v[next_state]

            if abs(v_old - v[state]) > delta: # Check if the value gap ∆ needs to be updated
                delta = abs(v_old - v[state])

        if delta == 0:
            return dict(list(v.items())[:-1]) # return all states, except the terminal state

pi = {
        's1': 'right',
        's2': 'suck',
        's3': 'up',
        's4': 'up'}

state_values = policy_evaluation(pi)
print("Policy evaluation state values:")
print(state_values)


###
# Q1.2: value iteration
###

"""
This value iteration method returns the optimal state values for all the states,
except the terminal states.
"""
def value_iteration():
    v = {} # initialise a dictionary 'v' to map states to state values
    for state in states:
        # foreach terminal state s_T, initialise its state value as its predefined reward v_π(s_T) = r_sT
        if state in terminal_states: 
            v[state] = terminal_reward
        # foreach state, set its state value v_π(s) to 0
        else: 
            v[state] = 0
            
    # Iterate until convergence
    while True:
        delta = 0 # value gap ∆ = 0
        # iterate over each state (except terminal states)
        for state in states[:-1]: # excluding last index, which is terminal state s5
            v_old = v[state] # copying old state value to v_old

            # Update state value
            max_value = float('-inf') # Any value will be bigger than negative infinity
            # Iterate through all applicable actions for the current state
            for action_index, action in enumerate(applicable_actions[states.index(state)]):
                # select the specific next state based on the current (state, action) pair being considered
                next_state = next_states[states.index(state)][action_index] 
                # select the specific reward based on the current (state, action) pair being considered
                reward = rewards[states.index(state)][action_index]

                # Calculate the resulting value for this (state, action) pair based on the next_state and reward
                resulting_value = reward + gamma * v[next_state]
                max_value = max(max_value, resulting_value) # compare current max with resulting_value and retain the larger value

            v[state] = max_value # update state value to the ultimate maximum value
            if abs(v_old - v[state]) > delta: # update delta if necessary
                delta = abs(v_old - v[state])

        if delta == 0:
            return dict(list(v.items())[:-1]) # return all states, except the terminal state

# Run value iteration
state_values = value_iteration()
print("\nValue iteration optimal state values:")
print(state_values)

###
# Stochastic environment value iteration
###
def stochastic_value_iteration():
    v = {} # initialise a dictionary 'v' to map states to state values
    for state in states:
        # foreach terminal state s_T, initialise its state value as its predefined reward v_π(s_T) = r_sT
        if state in terminal_states: 
            v[state] = terminal_reward
        # foreach state, set its state value v_π(s) to 0
        else: 
            v[state] = 0
            
    # Iterate until convergence
    while True:
        delta = 0 # value gap ∆ = 0
        # iterate over each state (except terminal states)
        for state in states[:-1]: # excluding last index, which is terminal state s5
            v_old = v[state] # copying old state value to v_old

            # Update state value
            max_value = float('-inf') # Any value will be bigger than negative infinity
            # Iterate through all applicable actions for the current state
            for action_index, action in enumerate(applicable_actions[states.index(state)]):
                # select the specific reward based on the current (state, action) pair being considered
                reward = rewards[states.index(state)][action_index]
                if (state == 's4' and action == 'suck'): 
                    success_state = 's5' # room is successfully cleaned
                    failure_state = state # room remains in s4, so it remains unclean
                    # 80% probability of success, 20% of failure
                    resulting_value = (0.8 * (reward + gamma * v[success_state])) + (0.2 * (reward + gamma * v[failure_state]))
                else:
                    # select the specific next state based on the current (state, action) pair being considered
                    next_state = next_states[states.index(state)][action_index] 
                    # Calculate the resulting value for this (state, action) pair based on the next_state and reward
                    resulting_value = reward + gamma * v[next_state]
                
                max_value = max(max_value, resulting_value) # compare current max with resulting_value and retain the larger value
            v[state] = max_value # update state value to the ultimate maximum value (after looping through every action)
            if abs(v_old - v[state]) > delta: # update delta if necessary
                delta = abs(v_old - v[state])
                    
            if delta < 0.001:
                return dict(list(v.items())[:-1]) # return all states, except the terminal state

# Run stochastic environment value iteration
stochastic_state_values = stochastic_value_iteration()
print("\nStochastic environment value iteration optimal state values:")
print(stochastic_state_values)

Policy evaluation state values:
{'s1': -0.9399999999999995, 's2': -0.9999999999999994, 's3': -0.8859999999999996, 's4': -0.9399999999999995}

Value iteration optimal state values:
{'s1': 0.5720000000000001, 's2': 0.68, 's3': 0.68, 's4': 0.8}

Stochastic environment value iteration optimal state values:
{'s1': 0.536323299872, 's2': 0.6403592220800001, 's3': 0.6403592220800001, 's4': 0.7560718444160002}
